# KoBigBird_Keyword_Extraction_Frequancy_Weighted

기존 방법에 빈도수에 따른 Frequancy_Weightt를 반영하는 모델

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

In [ ]:
#!pip install konlpy

### 불용어 제거 함수 정의

In [142]:
def remove_stopwords(nouns_list, stopword_set):
#     for noun in nouns_list[:]: # 원소를 삭제하는 과정에서 누락이 발생하기 때문에 [:]를 통해 리스트의 복사본을 주어야 함.
#         if noun in stopword_set:
#             nouns_list.remove(noun)
    arr_removed = [n for n in nouns_list if n not in stopword_set]
    return arr_removed

영어 불용어 받아오기

In [122]:
from nltk.corpus import stopwords as eng_stopwords

eng_stopwords_set = set(eng_stopwords.words('english'))

### 영어 단어 명사 추출 함수 정의

In [124]:
import nltk

from string import punctuation
import re

def get_nouns_eng(text):
    # 명사 추출 시 url 제거
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(url_pattern, '', text)
    url_pattern = re.compile(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)")
    text = re.sub(url_pattern, '', text)
    
    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
            
    return NN_words

## 불용어(stopwords) 사전 정의

두 개의 한국어 불용어 사전을 이용

In [101]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

770


### 명사구 추출 함수 정의

In [173]:
import konlpy
import nltk

def get_nouns_phrase(text):
    
    phrases = []
    words = konlpy.tag.Komoran().pos(text)

    # Define a chunk grammar, or chunking rules, then chunk
    grammar = """
    NP: {<N.*>*<Suffix>?}   # Noun phrase
    """
    parser = nltk.RegexpParser(grammar)
    chunks = parser.parse(words)


    print("\n# Print noun phrases only")
    for subtree in chunks.subtrees():
        if subtree.label()=='NP':
            phrases.append(' '.join((e[0] for e in list(subtree))))
    return phrases

### 각 형태소 분석기별 명사 추출 함수 정의

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [170]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter

def get_nouns_komoran(text, use_phrase=False):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    if use_phrase:
        phrases = get_nouns_phrase(text)
        nouns.extend(phrases)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    
    return nouns, counts

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    
    return nouns

In [6]:
# sample_nouns = get_nouns_komoran(sample[1]['body_text'])
# print(sample[1]['title'])
# print(sample[1]['body_text']+'\n')
# print(sample_nouns)

### 문장to문장 sts finetuning된 KoBigBird 모델 불러오기

In [7]:
from transformers import  AutoModel,AutoTokenizer
model = AutoModel.from_pretrained("./model/saved_model_epoch_79.pt")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

/home/user03/anaconda3/envs/koelectra/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BigBirdModel were not initialized from the model checkpoint at ./model/saved_model_epoch_79.pt and are newly initialized: ['pooler.weight', 'pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 데이터 불러오기

In [8]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch/brunch_dataset_100.jsonl") as f:
    for line in f.iter():
        sample.append(line)

In [9]:
sample[0]

{'title': '구글을 대체할 AI 챗봇 ChatGPT',
 'sub_title': '브런치의 미래\xa0',
 'body_text': '\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지능 챗봇을 선보였습니다. ChatGPT은 사용자와 주고받는 대화에서 질문에 답하도록 설계되어 있습니다. GPT는 질문에 답하는 것은 물론 정확하지 않은 전제에 대해 이의를 제기하기도 하고 부적절한 요청을 거부하는 등 마치 인격을 가진 인간과의 대화 구현이 가능합니다. \n\n이러한 GPT가 주목받게 된 것은 그 기능 자체의 뛰어남도 있지만 글로벌 기업인 구글의 모회사 CEO가 직접 언급하며 급속도로 기사화되었습니다. 구글에서 GPT를 경계하는 이유는 장기적으로 검색 엔진을 대체할 수 있을 것이라 염려 때문입니다. 현재의 검색 엔진은 사용자가 직접 검색 대상을 문법에 맞게 지시 내려야 합니다. 그러나 GPT는 대화형이기 때문에 누구나 손쉽게 이용할 수 있으며 더 다양한 정보를 (데이터)를 확보할 수 있습니다. \n\n제가 GPT에 관심을 갖게 된 것은 바로 GPT로 뛰어난 글을 작성할 수 있다는 부분 때문이었습니다. 실제로 해외 사용자들의 후기를  통해 GPT를 이용해 소설과 시를 썼다는 후기들을 손쉽게 볼 수 있습니다. \n*GPT는 영어로 커뮤니케이션하는 것이 더 원활합니다. 한국어도 지원하나 정보가 적으며 구동이 느립니다.\n\n캐나다 언론인이자 임상 심리학자이자인 조던피터슨(Jordan Bernt Peterson)은 GPT의 위험성에 대해 이야기하였습니다. 그의 날카로운 지적과 더불어 해당 영상을 본 사람들의 반응 또한 높은 통찰을 보여주고 있습니다. (자세한 내용은 아래 연결된 링크에서 확인 가능합니다)\n\n과거 인터넷의 발달로 우리는 편리한 생활을 누리게 되었지만 그만큼 축소된 부분도 존재합니다. 그 부분은 바로 생각에 대한 영역입니다. 세계적인 경영컨설턴트이자 IT 미래학자인 니콜라스 카가 그의 저서 <생각하지 않는 사람들>에서

In [10]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

## 최종 키워드 추출 함수(빈도수에 따른 가중치 반영)

In [205]:
def extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False):
    
    tokenized_nouns = {}
    for keyword in nouns:
        tokenized_nouns[keyword] = tokenizer(keyword, max_length=32, padding='max_length', return_tensors='pt')
    
    keyword_vectors = {}
    for keyword, tokenized in tokenized_nouns.items():
        output = model(**tokenized.to(device))[0][:,0,:]
        keyword_vectors[keyword] = output
        
    tokenized_title = tokenizer(title, max_length=128, padding="max_length", return_tensors='pt')
    tokenized_text = tokenizer(text, max_length=4096, padding="max_length", return_tensors='pt')

    title_vec = model(**tokenized_title.to(device))[0][:,0,:]
    body_text_vec = model(**tokenized_text.to(device))[0][:,0,:]
    
    if use_freq_w:
        total_words_counts = 0
        for word, count in counts.items():
            total_words_counts += count
        print(total_words_counts)
        
        freq_w = {}
    
        for keyword in nouns:
            if counts[keyword] <= 1:
                freq_w[keyword] = 0
                continue
            freq_w[keyword] = counts[keyword]/total_words_counts
        
        print(freq_w)
        for keyword, vec in keyword_vectors.items():
            body_text_vec = ((freq_w[keyword]* vec) + body_text_vec)
        
        body_text_vec = body_text_vec/total_words_counts
        
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    similarity = {}

    for keyword, vector in keyword_vectors.items():
        sim = cos(vector,  title_vec*(0.2) + body_text_vec*(0.8))
        similarity[keyword] = sim
        
    sorted_dict = sorted(similarity.items(), key = lambda item: item[1], reverse = True)
    
    # print(keyword_vectors)
    print(f"제목과 본문의 유사도 {cos(title_vec, body_text_vec)}\n")
    print(title+'\n')
    print(text+'\n')
    print(sorted_dict,'\n')
    print(nouns)

In [213]:
model.to('cuda')
model.eval()

sample_essay = sample[18003]

with torch.no_grad():
    print(sample_essay['keyword'])
    
    title = sample_essay['title']
    text = sample_essay['body_text']
    text = text.replace('\n','').replace('\t','').replace('\r','') # 코모란 에러 방지
    nouns, counts = get_nouns_komoran(text, use_phrase=False)
    print(counts)
    extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=True)

['환경보호', '에세이', '쓰레기']
Counter({'쓰레기': 35, '아이들': 12, '발견': 10, '담배꽁초': 10, '사람': 9, '오늘': 8, '캔': 7, '엄마': 5, '화단': 5, '재활용': 5, '노력': 4, '사용': 4, '환경': 4, '위생': 4, '생각': 4, '공원': 4, '전복': 4, '음료수': 4, '맥주': 4, '사탕': 4, '모임': 3, '실천': 3, '동네': 3, '준비물': 3, '준비': 3, '담': 3, '장갑': 3, '코스': 3, '아파트': 3, '껍데기': 3, '칫솔': 3, '하수구': 3, '사진': 3, '봉지': 3, '길': 3, '마음': 2, '포항': 2, '해변': 2, '일상생활': 2, '작년': 2, '코로나': 2, '플라스틱': 2, '보호': 2, '한쪽': 2, '듯이': 2, '보니': 2, '시작': 2, '처음': 2, '봉투': 2, '집게': 2, '근처': 2, '비교': 2, '담배': 2, '웰치스': 2, '포도': 2, '마트': 2, '전단지': 2, '비닐': 2, '뱃': 2, '갑': 2, '길거리': 2, '활용': 2, '동참': 2, '보람': 2, '내일': 2, '채널': 1, '큐': 1, '프로그램': 1, '시선': 1, '바닷가': 1, '가에': 1, '바이러스': 1, '발생': 1, '이후': 1, '온라인': 1, '주문': 1, '문': 1, '물품': 1, '배': 1, '송': 1, '배달': 1, '음식': 1, '우리나라': 1, '국민': 1, '사용량': 1, '폭발': 1, '반면': 1, '결성': 1, '가의': 1, '물티슈': 1, '대신': 1, '손수건': 1, '기저귀': 1, '텃밭': 1, '수세미': 1, '재배': 1, '재래시장': 1, '통': 1, '물건': 1, '장': 1, '제품': 1, '포장': 1, '회사': 1, '건의': 1, '메일': 

구현해야 할 점

* 단어를 배치단위로 입력하여 한번에 연산할 수 있게 변형
* 빈도수에 따른 가중치 변환
* 제목-본문 STS/NLI fine-tuning
* 단어-제목 STS/NLI fine-tuning
* 단어-본문 STS/NLI fine-tuning

### kr-wordrank와 비교

In [ ]:
from krwordrank.word import KRWordRank

min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
texts = [text]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

## 명사구 추출 코드

In [159]:
komoran = Komoran()
pos = komoran.pos(text)
print(pos)

[('지나', 'VV'), ('ㄴ', 'ETM'), ('주말', 'NNP'), ('막', 'NNG'), ('둥', 'NNB'), ('이', 'NNP'), ('가', 'JKS'), ('오랜만', 'NNG'), ('에', 'JKB'), ('부루마불', 'NNP'), ('을', 'JKO'), ('만지작거리', 'VV'), ('더니', 'EC'), ('목소리', 'NNG'), ('를', 'JKO'), ('높이', 'VV'), ('어', 'EC'), ('누나', 'NNG'), ('를', 'JKO'), ('부르', 'VV'), ('ㄴ다', 'EF'), ('.', 'SF'), ('"', 'SS'), ('누나', 'NNP'), ('아', 'NNP'), ('~', 'SO'), ('~', 'SO'), ('~', 'SO'), ('~', 'SO'), ('우리', 'NP'), ('부루마불', 'NNP'), ('하', 'VX'), ('ㄹ까', 'EF'), ('?', 'SF'), ('"', 'SS'), ('"', 'SS'), ('누', 'NNP'), ('"', 'SS'), ('는', 'JX'), ('들리', 'VV'), ('ㄹ락', 'EC'), ('말', 'VX'), ('락', 'EC'), ('"', 'SS'), ('나', 'NNP'), ('~', 'SO'), ('~', 'SO'), ('~', 'SO'), ('"', 'SS'), ('는', 'JX'), ('길', 'NNG'), (':', 'SP'), ('게', 'NNG'), ('부르', 'VV'), ('는', 'ETM'), ('목소리', 'NNG'), ('가', 'JKS'), ('참', 'MAG'), ('귀엽', 'VA'), ('고', 'EC'), ('도', 'JX'), ('정겹', 'VA'), ('다', 'EF'), ('.', 'SF'), ('나이', 'NNP'), ('를', 'JKO'), ('먹', 'VV'), ('을수록', 'EC'), ('위', 'NNG'), ('로', 'JKB'), ('십', 'NR'), ('년', 'NNB'),

In [160]:
komoran = Komoran()
pos = komoran.pos(text)

np_list = []
np = ''
cnt = 0
for tup in pos:
    if 'N' in tup[1]:
        np = np + tup[0]
        cnt += 1
        
    elif np != '' and cnt != 1:
        np_list.append(np)
        np = ''
        cnt = 0
    else:
        np = ''
        cnt = 0

np_list

['주말막둥이',
 '누나아',
 '우리부루마불',
 '십년아래',
 '십년',
 '키즈카페',
 '아이와의견',
 '학거리',
 '아홉살',
 '웃겨서콧방귀',
 '우리아이들도꼬',
 '듯이흥분',
 '오누이컨셉',
 '대사들',
 '전형적',
 '바른대답세트',
 '후요놈들',
 '귀여워누나',
 '성질',
 '막둥',
 '일명경고성기분',
 'ㅁ표시',
 '내기분',
 '조금씩',
 'ㅋㅋ이사람',
 '평소마음',
 '휴화산하나씩',
 '번씩',
 '기때문',
 '대인배',
 '무방비상태',
 '일기예보',
 '비소식',
 '오늘기분',
 '번씩',
 '기분상태',
 '상대방아',
 '깐죽거리',
 '미리미리조심',
 '내속']

In [199]:
import konlpy
import nltk

# POS tag a sentence
sentence = u'만 6세 이하의 초등학교 취학 전 자녀를 양육하기 위해서는'
words = konlpy.tag.Twitter().pos(text)

# Define a chunk grammar, or chunking rules, then chunk
grammar = """
NP: {<N.*>*}   # Noun phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)


print("\n# Print noun phrases only")
for subtree in chunks.subtrees():
    if subtree.label()=='NP':
        print(' '.join((e[0] for e in list(subtree))))



# Print noun phrases only
지난 주말 막둥이
만
부루마불
목소리
누나
누나
우리 부루마불
누
들릴락 말락
나
길
게
목소리
나이
위로 십 년 아래
십 년
바로 친구
어른
세계
달리 나이
위계 질서
것
일이
키즈 카페
처음
아이
의견
안
살짝 투닥거릴 때
은
너 몇 살이
난 아홉
나이
사건
종결
모습
보고
콧방귀
절로 나오니 말
우리 아이
두 살
차이
안 나
동생
누나
누나
나
선 무슨 큰일
난 듯이
막 하소연
듯
엄마
엄마
나 보고
속
뭐
말
겨우
누나
안
중재
속
픽
누나
소리
안방
유튜브
심취
누나
목소리
대답
오늘
오누이 컨셉
대사
드라마
법
듣기
전형
질문
예의
대답 세트
엄마 로서 참으로
잠시 후
놈
역시
무언가 의견
안
조그만 동생
누나
뭔가 꼼수
동생
목소리
위엄
목소리
마디
계속
내
성질
성질
강조
여유
가락
대사
나
웃음
막둥이
예고
일명 경고
기분
표시
일기예보
듯
내 기분
중이
조만간 성질
낼 수도
예고
오
이 사람
가끔
욱
때
그것
사람
우리
평소 마음
휴화산 하나
일상
다소
말
번
빵빵 터트려준다
왜
계속
참고
누르기 때문
사람
보이
참고
사람
참고 대인배
보이
인간
혼자
수가
존재
어찌
관계
수
존재
매사
꾹꾹
참고
경향
날
빵
마는 것
무방비 상태
상대
어이
더
싸움
것
일기 예보
내일
태풍
동반
비소 식이
것
말씀
좀
어머
안
좀 실망
저
조금 화가
것
오늘 기분
좀
등등
드라마
법
대사
번
나
기분 상태
좀 알
상대방
미리 예고
깐죽거릴 것
조금 참고
말
좀 참고 상대
좀 미리 미리
살짝 몸
사리
덩달아 내 속
휴화산
뻥
일도
말


In [196]:
print(words)

[('지난', 'Noun'), ('주말', 'Noun'), ('막둥이', 'Noun'), ('가', 'Josa'), ('오랜', 'Modifier'), ('만', 'Noun'), ('에', 'Josa'), ('부루마불', 'Noun'), ('을', 'Josa'), ('만지작거리더니', 'Adjective'), ('목소리', 'Noun'), ('를', 'Josa'), ('높여', 'Verb'), ('누나', 'Noun'), ('를', 'Josa'), ('부른다', 'Verb'), ('."', 'Punctuation'), ('누나', 'Noun'), ('아', 'Josa'), ('~~~~', 'Punctuation'), ('우리', 'Noun'), ('부루마불', 'Noun'), ('할까', 'Verb'), ('?""', 'Punctuation'), ('누', 'Noun'), ('"', 'Punctuation'), ('는', 'Verb'), ('들릴락', 'Noun'), ('말락', 'Noun'), ('"', 'Punctuation'), ('나', 'Noun'), ('~~~"', 'Punctuation'), ('는', 'Verb'), ('길', 'Noun'), (':', 'Punctuation'), ('게', 'Noun'), ('부르는', 'Verb'), ('목소리', 'Noun'), ('가', 'Josa'), ('참', 'Verb'), ('귀엽고도', 'Adjective'), ('정겹다', 'Adjective'), ('.', 'Punctuation'), ('나이', 'Noun'), ('를', 'Josa'), ('먹을수록', 'Verb'), ('위로', 'Noun'), ('십', 'Noun'), ('년', 'Noun'), ('아래', 'Noun'), ('로', 'Josa'), ('십', 'Noun'), ('년', 'Noun'), ('은', 'Josa'), ('바로', 'Noun'), ('친구', 'Noun'), ('를', 'Josa'), ('먹게', 'Verb')